In [1]:
# Get pandas and postgres to work together
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

In [2]:
# Postgres info to connect

connection_args = {
  "host": "twitchdata.chd4n5ul8muk.us-east-2.rds.amazonaws.com",
  "user": "postgres",
    "password":"FwwBFmleh65qYxKxDVb9",
  "port": 5432,
  "dbname": "twitchdata"
}


connection = pg.connect(**connection_args)

In [137]:
query = """
    SELECT game_name, game_genres, pic_url
    FROM general_view
    ;
"""

df = pd_sql.read_sql(query, connection)

In [138]:
df = df.drop_duplicates()


In [8]:
df.to_pickle('game_genre_pics.p')

In [3]:
# Final query after the database collection was stopped. This data was collected over the course of the week from 6/01 through 6/07.
query = "SELECT distinct(time_logged) FROM stream_data order by time_logged desc;"

pd_sql.read_sql(query, connection)

,time_logged
0,2019-06-07 03:02:52
1,2019-06-06 22:02:54
2,2019-06-06 21:02:55
3,2019-06-06 20:02:57
4,2019-06-06 19:04:09
5,2019-06-06 18:03:02
6,2019-06-06 17:04:01
7,2019-06-06 16:02:57
8,2019-06-06 15:03:54
9,2019-06-06 14:04:18


In [4]:
query = "SELECT COUNT(DISTINCT(stream_id)) FROM stream_data;"

pd_sql.read_sql(query, connection)

,count
0,350602


In [5]:
query = "SELECT COUNT(DISTINCT(game_id)) FROM game_information;"

pd_sql.read_sql(query, connection)

,count
0,532


In [6]:
query = "SELECT COUNT(DISTINCT(game_name)) FROM game_genres;"

pd_sql.read_sql(query, connection)

,count
0,1001


In [7]:
query = "SELECT Count(*) FROM stream_data;"

pd_sql.read_sql(query, connection)

,count
0,865196


## Querying the complete dataset for EDA as the new data comes in throughout the day. This query also created the general_view which we proceed to query from for all recommender workbooks.

In [8]:
query = "SELECT Count(*) FROM general_view;"

pd_sql.read_sql(query, connection)

,count
0,2954187


In [9]:
query = "SELECT * FROM general_view LIMIT 3;"

pd_sql.read_sql(query, connection)

,stream_id,user_name,game_id,stream_type,title,viewer_count,started_at,language,time_logged,game_name,pic_url,game_genres
0,34424458080,exxtrem_tv,27546,live,World of Tanks [GER] Relax,2,2019-06-06 09:10:12,de,2019-06-06 10:03:24,World of Tanks,https://static-cdn.jtvnw.net/ttv-boxart/World%...,Simulation
1,34424458080,exxtrem_tv,27546,live,World of Tanks [GER] Relax,2,2019-06-06 09:10:12,de,2019-06-06 10:03:24,World of Tanks,https://static-cdn.jtvnw.net/ttv-boxart/World%...,Action
2,34424458080,exxtrem_tv,27546,live,World of Tanks [GER] Relax,2,2019-06-06 09:10:12,de,2019-06-06 10:03:24,World of Tanks,https://static-cdn.jtvnw.net/ttv-boxart/World%...,Simulation


In [10]:
query = '''SELECT stream_id, user_id, user_name, stream_data.game_id, stream_type, title, viewer_count, started_at, language, time_logged, game_information.game_name, pic_url, game_genres.game_genres 
FROM stream_data 
LEFT JOIN game_information ON stream_data.game_id = game_information.game_id
LEFT JOIN game_genres ON game_information.game_name = game_genres.game_name;'''

complete_data = pd_sql.read_sql(query, connection)

In [11]:
complete_data.to_csv('final_general_view_06_07_19.csv', index = False)

## Getting the data for surprise recommender in the format to be funneled into the Surprise notebook.

In [ ]:
query = '''SELECT user_name, game_name, game_genres, language, started_at, viewer_count, max(viewer_count)
FROM general_view
GROUP BY user_name, game_name, game_genres, language, started_at, viewer_count
ORDER BY user_name DESC, max DESC'''

surprise_grid = pd_sql.read_sql(query, connection)

In [13]:
surprise_grid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1387492 entries, 0 to 1387491
Data columns (total 7 columns):
user_name       1387492 non-null object
game_name       1385813 non-null object
game_genres     1383564 non-null object
language        1387492 non-null object
started_at      1387492 non-null datetime64[ns]
viewer_count    1387492 non-null int64
max             1387492 non-null int64
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 74.1+ MB


In [14]:
surprise_grid.to_csv('final_game_user_grid_06_07_19.csv', index = False)


In [49]:
query = '''SELECT user_name, game_name, pic_url, max(viewer_count)
FROM general_view
GROUP BY user_name, game_name
ORDER BY user_name DESC, max DESC'''

minimal_grid = pd_sql.read_sql(query, connection)

DatabaseError: Execution failed on sql 'SELECT user_name, game_name, pic_url, max(viewer_count)
FROM general_view
GROUP BY user_name, game_name
ORDER BY user_name DESC, max DESC': column "general_view.pic_url" must appear in the GROUP BY clause or be used in an aggregate function
LINE 1: SELECT user_name, game_name, pic_url, max(viewer_count)
                                     ^


In [16]:
minimal_grid.to_csv('final_minimal_grid_06_07_19.csv', index = False)

In [17]:
surprise_grid.head()

,user_name,game_name,game_genres,language,started_at,viewer_count,max
0,龜狗,Grand Theft Auto V,Action,zh,2019-06-06 07:30:59,1735,1735
1,龜狗,Grand Theft Auto V,Open World,zh,2019-06-06 07:30:59,1735,1735
2,龜狗,Grand Theft Auto V,Driving/Racing Game,zh,2019-06-06 07:30:59,1735,1735
3,龜狗,Grand Theft Auto V,Open World,zh,2019-06-06 07:30:59,1589,1589
4,龜狗,Grand Theft Auto V,Action,zh,2019-06-06 07:30:59,1589,1589


In [18]:
minimal_grid.head()

,user_name,game_name,max
0,龜狗,Grand Theft Auto V,1735
1,龜狗,League of Legends,1058
2,龜狗,Just Chatting,467
3,龜仔_,PLAYERUNKNOWN'S BATTLEGROUNDS,23
4,龜仔_,Auto Chess,12


In [6]:
import pickle

In [139]:
df.head()

,game_name,game_genres,pic_url
0,World of Tanks,Action,https://static-cdn.jtvnw.net/ttv-boxart/World%...
1,World of Tanks,Simulation,https://static-cdn.jtvnw.net/ttv-boxart/World%...
112,The Elder Scrolls Online,Open World,https://static-cdn.jtvnw.net/ttv-boxart/The%20...
113,The Elder Scrolls Online,Stealth,https://static-cdn.jtvnw.net/ttv-boxart/The%20...
114,The Elder Scrolls Online,MMORPG,https://static-cdn.jtvnw.net/ttv-boxart/The%20...


In [20]:
surprise_grid.head()

,user_name,game_name,game_genres,language,started_at,viewer_count,max
0,龜狗,Grand Theft Auto V,Action,zh,2019-06-06 07:30:59,1735,1735
1,龜狗,Grand Theft Auto V,Open World,zh,2019-06-06 07:30:59,1735,1735
2,龜狗,Grand Theft Auto V,Driving/Racing Game,zh,2019-06-06 07:30:59,1735,1735
3,龜狗,Grand Theft Auto V,Open World,zh,2019-06-06 07:30:59,1589,1589
4,龜狗,Grand Theft Auto V,Action,zh,2019-06-06 07:30:59,1589,1589


In [292]:
x = df
x['game_genres'][x['game_genres'].str.contains('NONE') == True] = 'Other'
x['game_genres'][x['game_genres'].isnull()] = 'Other'
x['game_genres'][x['game_genres'].str.contains('RETROGAMEPLACEHOLDER')] = 'Other'

In [293]:
x.game_genres.unique()

array(['Action', 'Simulation', 'Open World', 'Stealth', 'MMORPG',
       'Card & Board Game', 'MOBA', 'Other', 'Sports Game',
       'Driving/Racing Game', 'Strategy', 'RPG', 'Indie Game',
       'Platformer', 'Shooter', 'Adventure Game', 'Series: Souls', 'FPS',
       'Puzzle', 'Rhythm & Music Game', 'Compilation', 'IRL', 'Creative',
       'RTS', 'Fighting', 'Horror', 'Roguelike', 'Gambling Game',
       'Survival', 'Hidden Objects', 'Flight Simulator', 'Metroidvania',
       'Educational Game', 'Point and Click', 'Visual Novel'],
      dtype=object)

In [390]:
type(x.head())

pandas.core.frame.DataFrame

In [295]:
x.loc[x['pic_url'].isnull()]

,game_name,game_genres,pic_url,pic_url2
5858,None,Other,None,None


In [296]:
x['pic_url2'] = x['pic_url'].apply(lambda y:  str(y).replace('-{width}x{height}',''))

In [297]:
x['pic_url2'][1]

'https://static-cdn.jtvnw.net/ttv-boxart/World%20of%20Tanks.jpg'

In [298]:
x.pic_url[0]

'https://static-cdn.jtvnw.net/ttv-boxart/World%20of%20Tanks-{width}x{height}.jpg'

In [299]:
x.head()

,game_name,game_genres,pic_url,pic_url2
0,World of Tanks,Action,https://static-cdn.jtvnw.net/ttv-boxart/World%...,https://static-cdn.jtvnw.net/ttv-boxart/World%...
1,World of Tanks,Simulation,https://static-cdn.jtvnw.net/ttv-boxart/World%...,https://static-cdn.jtvnw.net/ttv-boxart/World%...
112,The Elder Scrolls Online,Open World,https://static-cdn.jtvnw.net/ttv-boxart/The%20...,https://static-cdn.jtvnw.net/ttv-boxart/The%20...
113,The Elder Scrolls Online,Stealth,https://static-cdn.jtvnw.net/ttv-boxart/The%20...,https://static-cdn.jtvnw.net/ttv-boxart/The%20...
114,The Elder Scrolls Online,MMORPG,https://static-cdn.jtvnw.net/ttv-boxart/The%20...,https://static-cdn.jtvnw.net/ttv-boxart/The%20...


In [300]:
pickle.dump(x,open("surprise.p","wb"))

In [301]:
genre_dict = dict()

In [413]:
d = {}

for key,value in zip(x.game_name,x.game_genres):
    try:
        if value not in d[key]:
            d[key].append(value)
    except KeyError:
        d[key] = [value]

In [414]:
for key,value in zip(x.game_name,x.pic_url2):
    try:
        if value not in d[key]:
            d[key].append(value)
            continue
    except KeyError:
        d[key] = value

In [415]:
jpg_list = []
for item in d.values():
    jpg_list.append(item[-1])

In [417]:
d = {}

for key,value in zip(x.game_name,x.game_genres):
    try:
        if value not in d[key]:
            d[key].append(value)
    except KeyError:
        d[key] = [value]

In [418]:
list_key_value = [[k,v] for k,v in d.items()]

In [419]:
pickle.dump(list_key_value, open( "surprise.p", "wb" ) )

In [420]:
pd.DataFrame(jpg_list).head()

,0
0,https://static-cdn.jtvnw.net/ttv-boxart/World%...
1,https://static-cdn.jtvnw.net/ttv-boxart/The%20...
2,https://static-cdn.jtvnw.net/ttv-boxart/Poker.jpg
3,https://static-cdn.jtvnw.net/ttv-boxart/Heroes...
4,https://static-cdn.jtvnw.net/ttv-boxart/BanG%2...


In [506]:
columns = ['game','genres']
test = pd.DataFrame(list_key_value, columns=columns)

In [507]:
test['link'] = jpg_list

In [508]:
test.head()

,game,genres,link
0,World of Tanks,"[Action, Simulation]",https://static-cdn.jtvnw.net/ttv-boxart/World%...
1,The Elder Scrolls Online,"[Open World, Stealth, MMORPG]",https://static-cdn.jtvnw.net/ttv-boxart/The%20...
2,Poker,[Card & Board Game],https://static-cdn.jtvnw.net/ttv-boxart/Poker.jpg
3,Heroes of the Storm,[MOBA],https://static-cdn.jtvnw.net/ttv-boxart/Heroes...
4,BanG Dream! Girls Band Party!,[Other],https://static-cdn.jtvnw.net/ttv-boxart/BanG%2...


In [509]:
pliz = test

In [553]:
pliz['genres'][0]

['Action', 'Simulation']

In [608]:
k=[]
for item in pliz['genres']:
    item = str(item).strip(', ')
    item = re.sub(r'\W+', ' ',str(item).replace(' ',''))
#     str(item).replace(' ','')
    k.append(item) 
#     item = re.sub(r'\W+', ' ',str(item))
#     print(item)
print(k[0:5])

[' Action Simulation ', ' OpenWorld Stealth MMORPG ', ' Card BoardGame ', ' MOBA ', ' Other ']


In [611]:
test['html_genres']=k
# for item in k:
#     print(item)

In [612]:
test.head()

,game,genres,link,html_genres
0,World of Tanks,"[Action, Simulation]",https://static-cdn.jtvnw.net/ttv-boxart/World%...,Action Simulation
1,The Elder Scrolls Online,"[Open World, Stealth, MMORPG]",https://static-cdn.jtvnw.net/ttv-boxart/The%20...,OpenWorld Stealth MMORPG
2,Poker,[Card & Board Game],https://static-cdn.jtvnw.net/ttv-boxart/Poker.jpg,Card BoardGame
3,Heroes of the Storm,[MOBA],https://static-cdn.jtvnw.net/ttv-boxart/Heroes...,MOBA
4,BanG Dream! Girls Band Party!,[Other],https://static-cdn.jtvnw.net/ttv-boxart/BanG%2...,Other


In [613]:
for item in test.iterrows():
    print(f'<div class="mix {item[1][1]}"><img src="{item[1][2]}" alt="{item[1][0]}">{item[1][0]}</div>')

(0, game                                              World of Tanks
genres                                      [Action, Simulation]
link           https://static-cdn.jtvnw.net/ttv-boxart/World%...
html_genres                                   Action Simulation 
Name: 0, dtype: object)
(1, game                                    The Elder Scrolls Online
genres                             [Open World, Stealth, MMORPG]
link           https://static-cdn.jtvnw.net/ttv-boxart/The%20...
html_genres                            OpenWorld Stealth MMORPG 
Name: 1, dtype: object)
(2, game                                                       Poker
genres                                       [Card & Board Game]
link           https://static-cdn.jtvnw.net/ttv-boxart/Poker.jpg
html_genres                                      Card BoardGame 
Name: 2, dtype: object)
(3, game                                         Heroes of the Storm
genres                                                    [MOBA]
li

Name: 333, dtype: object)
(334, game                                               Overcooked! 2
genres                                                   [Other]
link           https://static-cdn.jtvnw.net/ttv-boxart/Overco...
html_genres                                               Other 
Name: 334, dtype: object)
(335, game                                    Hector: Badge of Carnage
genres                                                   [Other]
link           https://static-cdn.jtvnw.net/ttv-boxart/./Hect...
html_genres                                               Other 
Name: 335, dtype: object)
(336, game                                              Dirt Rally 2.0
genres                                     [Driving/Racing Game]
link           https://static-cdn.jtvnw.net/ttv-boxart/Dirt%2...
html_genres                                  Driving RacingGame 
Name: 336, dtype: object)
(337, game                                          Guardians of Ember
genres                     

In [546]:
import re

# for item in pliz.iterrows():
#     item[1][1] = 'WORKS'
#     print(item[1][1])
for item in pliz.iterrows():
    for word in item[1][1]:
        word = re.sub(r'\W', '', str(word))
        print(word)
    print(item[1][1])
#     print(re.sub(r'\W', '', str(item[1][1])))
    
#     print(' '.join(i for i in item[1][1] if i.isalpha()))
#     print(''.join(ch for ch in item[1][1] if ch.isalnum()))
#     item[1][1] = re.sub(r'\W+', ' ', str(item[1][1]))
# #     new_item = [i for i in item[1][1] if i.isalpha()]
#     print(item[1][1])
#     print ([''.join(i) for i in item[1][1]])
#     new_item = ''.join([i for i in this_item if i.isalpha()])    
#     item[1][1] = ''.
#     string.join(item[1][1])
#     this_item = item[1][1].replace(" ","")
#     print(this_item)
#     new_item = ''.join([i for i in this_item if i.isalpha()])
#     print(f'<button type="button" class="control" data-toggle=".{new_item}">{item}</button>')    

Action
Simulation
['Action', 'Simulation']
OpenWorld
Stealth
MMORPG
['Open World', 'Stealth', 'MMORPG']
CardBoardGame
['Card & Board Game']
MOBA
['MOBA']
Other
['Other']
SportsGame
['Sports Game']
DrivingRacingGame
SportsGame
Action
['Driving/Racing Game', 'Sports Game', 'Action']
Strategy
RPG
['Strategy', 'RPG']
IndieGame
Platformer
Action
['Indie Game', 'Platformer', 'Action']
OpenWorld
Shooter
RPG
['Open World', 'Shooter', 'RPG']
Platformer
AdventureGame
['Platformer', 'Adventure Game']
SeriesSouls
Action
RPG
['Series: Souls', 'Action', 'RPG']
RPG
Action
['RPG', 'Action']
Shooter
FPS
['Shooter', 'FPS']
FPS
Puzzle
AdventureGame
['FPS', 'Puzzle', 'Adventure Game']
Other
['Other']
Action
RPG
Simulation
['Action', 'RPG', 'Simulation']
Other
['Other']
MMORPG
RPG
['MMORPG', 'RPG']
RhythmMusicGame
['Rhythm & Music Game']
Action
RPG
Strategy
['Action', 'RPG', 'Strategy']
Other
['Other']
Shooter
Action
['Shooter', 'Action']
MOBA
['MOBA']
OpenWorld
Action
Platformer
['Open World', 'Action', '

In [591]:
pliz.head()

,game,genres,link
0,World of Tanks,"[Action, Simulation]",https://static-cdn.jtvnw.net/ttv-boxart/World%...
1,The Elder Scrolls Online,"[Open World, Stealth, MMORPG]",https://static-cdn.jtvnw.net/ttv-boxart/The%20...
2,Poker,[Card & Board Game],https://static-cdn.jtvnw.net/ttv-boxart/Poker.jpg
3,Heroes of the Storm,[MOBA],https://static-cdn.jtvnw.net/ttv-boxart/Heroes...
4,BanG Dream! Girls Band Party!,[Other],https://static-cdn.jtvnw.net/ttv-boxart/BanG%2...


In [615]:
for item in test.iterrows():
    print(f'<div class="mix {item[1][3]}"><img src="{item[1][2]}" alt="{item[1][0]}">{item[1][0]}</div>')

<div class="mix  Action Simulation "><img src="https://static-cdn.jtvnw.net/ttv-boxart/World%20of%20Tanks.jpg" alt="World of Tanks">World of Tanks</div>
<div class="mix  OpenWorld Stealth MMORPG "><img src="https://static-cdn.jtvnw.net/ttv-boxart/The%20Elder%20Scrolls%20Online.jpg" alt="The Elder Scrolls Online">The Elder Scrolls Online</div>
<div class="mix  Card BoardGame "><img src="https://static-cdn.jtvnw.net/ttv-boxart/Poker.jpg" alt="Poker">Poker</div>
<div class="mix  MOBA "><img src="https://static-cdn.jtvnw.net/ttv-boxart/Heroes%20of%20the%20Storm.jpg" alt="Heroes of the Storm">Heroes of the Storm</div>
<div class="mix  Other "><img src="https://static-cdn.jtvnw.net/ttv-boxart/BanG%20Dream%21%20Girls%20Band%20Party%21.jpg" alt="BanG Dream! Girls Band Party!">BanG Dream! Girls Band Party!</div>
<div class="mix  SportsGame "><img src="https://static-cdn.jtvnw.net/ttv-boxart/NBA%202K19.jpg" alt="NBA 2K19">NBA 2K19</div>
<div class="mix  Driving RacingGame SportsGame Action "><im

In [372]:
# x = ', '.join(test['genres'])
for item in test['genres']:
    f = ', '.join(item)
    print(f)

Action, Simulation
Open World, Stealth, MMORPG
Card & Board Game
MOBA
Other
Sports Game
Driving/Racing Game, Sports Game, Action
Strategy, RPG
Indie Game, Platformer, Action
Open World, Shooter, RPG
Platformer, Adventure Game
Series: Souls, Action, RPG
RPG, Action
Shooter, FPS
FPS, Puzzle, Adventure Game
Other
Action, RPG, Simulation
Other
MMORPG, RPG
Rhythm & Music Game
Action, RPG, Strategy
Other
Shooter, Action
MOBA
Open World, Action, Platformer
Rhythm & Music Game
Action, Puzzle, Compilation
IRL, Creative
Other
Other
Action
IRL
Action
Card & Board Game, RTS, Strategy
RPG, Adventure Game, MMORPG
Action
Platformer, Fighting
Shooter, FPS, RPG
MMORPG, Adventure Game, Action
Indie Game, Fighting
Other
RPG
Card & Board Game
Action
RPG, Strategy
Creative, IRL
Card & Board Game
Simulation
Simulation
Indie Game, Horror
Shooter, FPS, Action
Fighting
Action
MOBA
Shooter
Shooter, FPS
IRL
Open World, Driving/Racing Game, Action
Card & Board Game
MMORPG
MOBA
Adventure Game, Open World, Action
F

In [364]:
final_list

[0      World of Tankshttps://static-cdn.jtvnw.net/ttv...
 1      The Elder Scrolls Onlinehttps://static-cdn.jtv...
 2      Pokerhttps://static-cdn.jtvnw.net/ttv-boxart/P...
 3      Heroes of the Stormhttps://static-cdn.jtvnw.ne...
 4      BanG Dream! Girls Band Party!https://static-cd...
 5      NBA 2K19https://static-cdn.jtvnw.net/ttv-boxar...
 6      Rocket Leaguehttps://static-cdn.jtvnw.net/ttv-...
 7      Heroes of Might and Magic III: The Shadow of D...
 8      Cupheadhttps://static-cdn.jtvnw.net/ttv-boxart...
 9      Tom Clancy's The Division 2https://static-cdn....
 10     Super Mario 64https://static-cdn.jtvnw.net/ttv...
 11     Dark Souls IIIhttps://static-cdn.jtvnw.net/ttv...
 12     Monster Hunter Worldhttps://static-cdn.jtvnw.n...
 13     Battalion 1944https://static-cdn.jtvnw.net/ttv...
 14     Draugenhttps://static-cdn.jtvnw.net/ttv-boxart...
 15     Retrohttps://static-cdn.jtvnw.net/ttv-boxart/R...
 16     VRChathttps://static-cdn.jtvnw.net/ttv-boxart/...
 17     Trover

In [357]:
for item in test['genres']:
    print(item)

['Action', 'Simulation']
['Open World', 'Stealth', 'MMORPG']
['Card & Board Game']
['MOBA']
['Other']
['Sports Game']
['Driving/Racing Game', 'Sports Game', 'Action']
['Strategy', 'RPG']
['Indie Game', 'Platformer', 'Action']
['Open World', 'Shooter', 'RPG']
['Platformer', 'Adventure Game']
['Series: Souls', 'Action', 'RPG']
['RPG', 'Action']
['Shooter', 'FPS']
['FPS', 'Puzzle', 'Adventure Game']
['Other']
['Action', 'RPG', 'Simulation']
['Other']
['MMORPG', 'RPG']
['Rhythm & Music Game']
['Action', 'RPG', 'Strategy']
['Other']
['Shooter', 'Action']
['MOBA']
['Open World', 'Action', 'Platformer']
['Rhythm & Music Game']
['Action', 'Puzzle', 'Compilation']
['IRL', 'Creative']
['Other']
['Other']
['Action']
['IRL']
['Action']
['Card & Board Game', 'RTS', 'Strategy']
['RPG', 'Adventure Game', 'MMORPG']
['Action']
['Platformer', 'Fighting']
['Shooter', 'FPS', 'RPG']
['MMORPG', 'Adventure Game', 'Action']
['Indie Game', 'Fighting']
['Other']
['RPG']
['Card & Board Game']
['Action']
['RPG', 

In [272]:
pickle.dump(test, open( "rm_genre_pic_list.p", "wb" ) )

In [ ]:
# <div class="mix FPS"><img src="https://static-cdn.jtvnw.net/ttv-boxart/NBA%202K19.jpg" alt="Italian Trulli">GAME2</div>